In [3]:
# /*==========================================================================================*\
# **                        _           _ _   _     _  _         _                            **
# **                       | |__  _   _/ | |_| |__ | || |  _ __ | |__                         **
# **                       | '_ \| | | | | __| '_ \| || |_| '_ \| '_ \                        **
# **                       | |_) | |_| | | |_| | | |__   _| | | | | | |                       **
# **                       |_.__/ \__,_|_|\__|_| |_|  |_| |_| |_|_| |_|                       **
# \*==========================================================================================*/


# -----------------------------------------------------------------------------------------------
# Author: Bùi Tiến Thành - Tien-Thanh Bui (@bu1th4nh)
# Title: main_mofa2.ipynb
# Date: 2024/11/07 12:46:46
# Description: Baseline implementation for MOFA2 against our model
# 
# (c) 2024 bu1th4nh. All rights reserved. 
# Written with dedication in the University of Central Florida, EPCOT and the Magic Kingdom.
# -----------------------------------------------------------------------------------------------

import sys
sys.path.insert(1, '../../')


import mlflow
import pymongo
import logging
import numpy as np
import mofax as mfx
import pandas as pd
from tqdm import tqdm
from s3fs import S3FileSystem
from mofapy2.run.entry_point import entry_point
from typing import List, Dict, Any, Tuple, Union, Literal
from downstream.classification import evaluate_one_target

tqdm.pandas()
mlflow.set_tracking_uri('http://localhost:6969')




key = 'bu1th4nh'
secret = 'ariel.anna.elsa'
endpoint_url = 'http://localhost:19000'

s3 = S3FileSystem(
    anon=False, 
    endpoint_url=endpoint_url,
    key=key,
    secret=secret,
    use_ssl=False
)
storage_option = {
    'key': key,
    'secret': secret,
    'endpoint_url': endpoint_url,
}

mongo = pymongo.MongoClient(
    host='mongodb://localhost',
    port=27017,
    username='bu1th4nh',
    password='ariel.anna.elsa',
)
mongo_db = mongo['SimilarSampleCrossOmicNMF']


configs = [
    ('BreastCancer/processed_crossOmics', 'BreastCancer', 'brca', 'BRCA', 'SimilarSampleCrossOmicNMFv3'),
    ('LungCancer/processed', 'LungCancer', 'luad', 'LUAD', 'SimilarSampleCrossOmicNMFv3_LUAD'),
    ('OvarianCancer/processed', 'OvarianCancer', 'ov', 'OV', 'SimilarSampleCrossOmicNMFv3_OV'),
]
mofa_latent_dims = 15

def find_run(collection, run_id: str, target_id: str): return collection.find_one({'run_id': run_id, 'target_id': target_id})

In [4]:
for ds_name, general_data_name, res_folder, mongo_collection, mlf_experiment_name in configs:
    DATA_PATH = f's3://datasets/{ds_name}'
    DATA_PATH = f's3://datasets/{ds_name}'
    TARG_PATH = f's3://datasets/{general_data_name}/clinical_testdata'
    DR_RES_PATH = f's3://results/SimilarSampleCrossOmicNMF/{res_folder}/baseline_MOFA2'
    miRNA = pd.read_parquet(f"{DATA_PATH}/miRNA.parquet", storage_options=storage_option)
    mRNA = pd.read_parquet(f"{DATA_PATH}/mRNA.parquet", storage_options=storage_option)

    mlflow.set_experiment(mlf_experiment_name)
    collection = mongo_db[mongo_collection]

    # miRNA.head()
    print("Dataset: ", ds_name.split('/')[0])
    print("miRNA")
    print(f"Sample size: {miRNA.shape[1]}")
    print(f"Feature size: {miRNA.shape[0]}")
    print("mRNA")
    print(f"Sample size: {mRNA.shape[1]}")
    print(f"Feature size: {mRNA.shape[0]}")

    display(miRNA.head())
    display(mRNA.head())

    data_mat = [[miRNA.T.values], [mRNA.T.values]]

    Ariel = entry_point()
    Ariel.set_data_matrix(
        data_mat, 
        likelihoods=['gaussian', 'gaussian'], 
        views_names=['miRNA', 'mRNA'],
        features_names=[miRNA.index, mRNA.index],
        samples_names=[miRNA.columns],
    )

    Ariel.set_model_options(
        factors=mofa_latent_dims
    )

    Ariel.set_train_options(
        convergence_mode = "fast",
    )

    Ariel.build()
    Ariel.run()

    Ariel.save("output.hdf5")
    Belle = mfx.mofa_model("output.hdf5").get_factors(factors=range(mofa_latent_dims), df=True)
    
    s3.mkdirs(DR_RES_PATH, exist_ok=True)
    Belle.to_parquet(f"{DR_RES_PATH}/H.parquet", storage_options=storage_option)
    
    run_id = s3.open(f"{DR_RES_PATH}/run_id.txt", 'r').read() if s3.exists(f"{DR_RES_PATH}/run_id.txt") else None

    
    with mlflow.start_run(run_id=run_id) if run_id is not None else mlflow.start_run(run_name='baseline_MOFA2'):


        mlflow.log_param("Number of omics layers", 2)
        mlflow.log_param("Omics layers feature size", [mRNA.shape[0], miRNA.shape[0]])
        mlflow.log_param("Sample size", miRNA.shape[1])
        mlflow.log_param("Latent size", mofa_latent_dims)


        if run_id is None: 
            run_id = mlflow.active_run().info.run_id
            with s3.open(f"{DR_RES_PATH}/run_id.txt", 'w') as f:
                f.write(run_id)
        H = Belle.copy(deep=True)
        target_folders = [f's3://{a}' for a in s3.ls(TARG_PATH)]

        for target_folder in target_folders:
            # Retrieve test data
            target_id = str(target_folder.split('/')[-1]).split('.')[0]
            if find_run(collection, run_id, target_id) is not None:
                logging.info(f"Run {run_id} on dataset {target_id} already exists. Skipping")
                continue
            test_data = pd.read_parquet(target_folder, storage_options=storage_option)

            # Evaluate
            result_pack = evaluate_one_target(H, testdata = test_data, methods_list = ["Logistic Regression", "Random Forest"], target = target_id)

            # Load to staging package
            data_pack = {
                'run_id': run_id,
                'target_id': target_id,
                'summary': {}
            }
            for method in result_pack.keys():
                data_pack[method] = result_pack[method].to_dict(orient='index')

                for metric in result_pack[method].columns:
                    if str(metric).isupper():
                        # Assume all metrics are upper case-noted columns
                        data_pack['summary'][f'{method} Mean {metric}'] = np.mean(result_pack[method][metric].values)
                        data_pack['summary'][f'{method} Median {metric}'] = np.median(result_pack[method][metric].values)
                        data_pack['summary'][f'{method} Std {metric}'] = np.std(result_pack[method][metric].values)
                        data_pack['summary'][f'{method} Max {metric}'] = np.max(result_pack[method][metric].values)
                        data_pack['summary'][f'{method} Min {metric}'] = np.min(result_pack[method][metric].values)

            # Log to MLFlow
            for key in data_pack['summary'].keys():
                if 'Mean AUROC' in key: mlflow.log_metric(f'{target_id} {key}', data_pack['summary'][key])
                if 'Mean MCC' in key: mlflow.log_metric(f'{target_id} {key}', data_pack['summary'][key])
        
        
            # Save to MongoDB
            collection.insert_one(data_pack)

    

Dataset:  BreastCancer
miRNA
Sample size: 830
Feature size: 277
mRNA
Sample size: 830
Feature size: 10480


Sample,TCGA-3C-AAAU-01,TCGA-3C-AALI-01,TCGA-3C-AALJ-01,TCGA-3C-AALK-01,TCGA-4H-AAAK-01,TCGA-5L-AAT0-01,TCGA-5T-A9QA-01,TCGA-A1-A0SB-01,TCGA-A1-A0SD-01,TCGA-A1-A0SF-01,...,TCGA-UL-AAZ6-01,TCGA-UU-A93S-01,TCGA-V7-A7HQ-01,TCGA-W8-A86G-01,TCGA-WT-AB41-01,TCGA-WT-AB44-01,TCGA-XX-A899-01,TCGA-XX-A89A-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01
miRNA_ID,,,,,,,,,,,,,,,,,,,,,
MIMAT0000062,15.124849,14.916849,15.013644,15.143310,15.413594,15.314889,15.720944,15.602910,15.603126,14.341908,...,13.398294,14.823079,15.066375,15.711691,15.038787,15.368482,16.037338,15.684479,14.967621,15.354657
MIMAT0000063,14.591428,14.509958,13.417786,14.665232,14.436443,14.574856,15.095598,16.502043,15.637101,14.319066,...,11.393023,13.499891,13.603028,15.207085,13.906266,14.512249,14.337599,14.196586,14.318915,14.114179
MIMAT0000064,8.401837,9.638816,9.304503,11.504435,11.683468,11.131379,9.188562,13.346309,11.401253,11.897654,...,7.357762,10.823890,10.657737,12.464927,11.230791,11.919849,12.358977,12.680574,11.966794,10.369241
MIMAT0000065,7.482535,7.891342,8.116467,7.179990,7.293012,6.993267,8.001244,8.663774,7.741665,7.088881,...,8.175810,7.363713,7.198927,7.636294,8.190430,7.609878,6.659060,7.943300,7.147604,8.743464
MIMAT0000066,10.499502,9.121162,11.389719,10.351892,10.727595,9.694004,11.128364,9.305857,10.536886,9.265151,...,9.941453,10.211138,11.265103,10.740918,10.401965,10.526240,11.082142,11.307832,9.882271,10.915322


Sample,TCGA-3C-AAAU-01,TCGA-3C-AALI-01,TCGA-3C-AALJ-01,TCGA-3C-AALK-01,TCGA-4H-AAAK-01,TCGA-5L-AAT0-01,TCGA-5T-A9QA-01,TCGA-A1-A0SB-01,TCGA-A1-A0SD-01,TCGA-A1-A0SF-01,...,TCGA-UL-AAZ6-01,TCGA-UU-A93S-01,TCGA-V7-A7HQ-01,TCGA-W8-A86G-01,TCGA-WT-AB41-01,TCGA-WT-AB44-01,TCGA-XX-A899-01,TCGA-XX-A89A-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01
mRNA_ID,,,,,,,,,,,,,,,,,,,,,
A1BG,7.6300,7.8971,8.7287,7.5851,8.0762,7.6779,8.3001,5.6496,7.1629,8.3532,...,5.4968,8.3353,10.0127,9.0608,6.9166,8.8835,7.8088,8.0008,8.7831,7.9619
A1CF,0.0000,0.0000,0.9310,0.0000,0.5115,0.0000,0.6069,0.0000,0.0000,0.0000,...,0.4846,0.0000,0.0000,0.5603,0.0000,0.0000,0.0000,1.7492,0.0000,0.0000
A4GNT,3.2655,0.6265,0.0000,0.0000,0.5115,2.3523,2.2207,0.9268,0.7326,0.8928,...,0.4846,0.0000,0.6922,1.5353,0.0000,0.8596,0.8379,1.9389,0.0000,0.0000
AAGAB,10.3458,11.7904,10.4113,10.7181,10.8166,10.8867,11.3361,9.7309,10.9316,9.9775,...,11.3978,10.3083,11.7017,10.1852,10.5008,9.6498,10.5380,10.5811,10.6573,10.5149
AAK1,9.0177,9.7301,9.1534,9.5093,9.4079,9.8875,8.4332,10.0352,10.1115,9.3754,...,9.5825,9.6860,8.4134,9.5813,9.3072,8.9643,9.9129,9.8219,9.3383,9.0306



        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Groups names not provided, using default naming convention:
- group1, group2, ..., groupG

Successfully loaded view='miRNA' group='group0' with N=830 samples and D=277 features...
Successfully loaded view='mRNA' group='group0' with N=830 samples and D=10480 features...


Model options:
- Automatic Relevance Determination prior on the factors: False
- Automatic Relevance Determin

Evaluating target TN on testdata: 100%|██████████| 200/200 [00:26<00:00,  7.48it/s]


...

Sample,TCGA-04-1348-01,TCGA-04-1357-01,TCGA-04-1362-01,TCGA-04-1364-01,TCGA-04-1365-01,TCGA-04-1514-01,TCGA-04-1519-01,TCGA-09-0364-01,TCGA-09-0366-01,TCGA-09-0367-01,...,TCGA-61-2102-01,TCGA-61-2104-01,TCGA-61-2109-01,TCGA-61-2110-01,TCGA-61-2111-01,TCGA-61-2113-01,TCGA-OY-A56P-01,TCGA-OY-A56Q-01,TCGA-VG-A8LO-01,TCGA-WR-A838-01
miRNA_ID,,,,,,,,,,,,,,,,,,,,,
MIMAT0000062,16.471727,15.836641,16.305966,15.499536,15.483672,14.104003,16.227724,12.948855,14.411871,15.619096,...,15.617217,16.377085,16.520540,16.964991,16.300330,16.234423,15.857841,14.990592,15.265951,15.242781
MIMAT0000063,16.809931,16.149256,16.074727,15.120727,16.358887,14.235049,18.008363,12.157711,15.683184,16.867409,...,13.454406,16.062571,15.897134,15.263910,15.395286,15.864365,13.496025,11.387720,13.654316,14.003587
MIMAT0000064,13.674967,14.333572,13.308891,12.106964,12.734394,11.355395,14.440998,11.862731,13.153581,14.350413,...,13.898993,15.867024,15.271230,14.561708,12.296455,13.883136,13.225713,12.350666,12.124840,12.979874
MIMAT0000065,9.492601,8.861313,9.307303,7.502286,9.096564,7.092028,7.995523,5.351382,7.805330,7.989392,...,5.892084,8.899942,9.624664,6.839433,8.733292,9.027311,8.222986,6.206050,9.117540,9.283728
MIMAT0000066,11.541527,12.737216,11.697319,11.758332,11.443721,10.580158,11.399947,10.583674,10.369197,10.583642,...,12.934612,12.367565,13.241546,12.969504,13.161630,14.066687,11.392159,10.859500,11.181756,11.395201


Sample,TCGA-04-1348-01,TCGA-04-1357-01,TCGA-04-1362-01,TCGA-04-1364-01,TCGA-04-1365-01,TCGA-04-1514-01,TCGA-04-1519-01,TCGA-09-0364-01,TCGA-09-0366-01,TCGA-09-0367-01,...,TCGA-61-2102-01,TCGA-61-2104-01,TCGA-61-2109-01,TCGA-61-2110-01,TCGA-61-2111-01,TCGA-61-2113-01,TCGA-OY-A56P-01,TCGA-OY-A56Q-01,TCGA-VG-A8LO-01,TCGA-WR-A838-01
mRNA_ID,,,,,,,,,,,,,,,,,,,,,
A1BG,6.0762,6.0567,5.4142,7.5549,4.6398,5.0795,5.5650,7.3563,6.4847,7.5979,...,6.0157,6.6460,5.2990,7.1433,6.5806,6.1586,5.4651,5.1163,4.8560,4.1051
A1CF,0.0000,0.0000,0.4125,0.0000,0.0000,0.0000,0.0000,0.9286,0.0000,0.0000,...,0.0000,0.0000,0.0000,1.5389,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A4GNT,0.6209,0.0000,0.0000,0.0000,0.6805,0.8981,0.0000,1.2358,1.6074,1.6406,...,1.1372,1.0457,2.8477,2.6417,0.6784,0.4323,3.8438,1.4147,0.4028,1.3436
AAGAB,9.9738,9.3773,9.2942,10.1638,10.3303,9.7287,8.9198,10.2383,10.1968,8.9670,...,9.8833,9.6935,10.3546,9.1313,9.6212,10.2996,9.3087,9.3349,10.0340,10.1418
AAK1,10.9220,10.4399,9.6598,10.2647,10.1384,9.2073,11.0253,9.1123,10.9995,10.8229,...,10.5535,11.0349,10.6810,10.0577,11.2380,11.4476,12.9036,11.4020,12.0201,10.5262



        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Groups names not provided, using default naming convention:
- group1, group2, ..., groupG

Successfully loaded view='miRNA' group='group0' with N=304 samples and D=274 features...
Successfully loaded view='mRNA' group='group0' with N=304 samples and D=10479 features...


Model options:
- Automatic Relevance Determination prior on the factors: False
- Automatic Relevance Determin

Evaluating target survival_36_36 on testdata: 100%|██████████| 100/100 [00:13<00:00,  7.31it/s]


🏃 View run baseline_MOFA2 at: http://localhost:6969/#/experiments/7/runs/1d86a72dbff042cb9aed6d85a552793c
🧪 View experiment at: http://localhost:6969/#/experiments/7
